# RAG evaluation: 
- Retrieval: Primarly examine the recall of the RAG system.
- Generation: Secondarily examine the generation quality of the RAG system.


### Retrieval
Context Precision & Context Recall

### Generation
Faithfulness


# 1. Evaluating Retrieval

In [1]:
import sys
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

current_dir = Path.cwd().parent
parent_dir = current_dir.parent

sys.path.append(str(parent_dir))
from backend.brain.document_processing import DocumentProcessor
from research.utils import QuestionAnswerPair, generate_question, save_qa_pairs, load_qa_pairs
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore


# load & chunk data 
pdf_path = (
        parent_dir / "backend" / "docs" / "Bitcoin - A Peer-to-Peer Electronic Cash System.pdf"
    )

processor = DocumentProcessor()
pages = processor.load_pdf(pdf_path)
chunks = processor.chunk_docs(pages)

# create embeddings 
vector_store = InMemoryVectorStore(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)
vector_store.add_documents(chunks)

# generate questions
qa_pairs = []

for idx, doc in enumerate(chunks[:20]):
    question = generate_question(doc.page_content)
    qa_pairs.append(
        QuestionAnswerPair(
            question=question,
            source_chunk=doc.page_content,
            chunk_index=idx,
        )
    )



In [6]:
# save qa_pairs to file
parent_dir = Path.cwd().parent
data_dir = parent_dir / "research" / "data"
save_qa_pairs(qa_pairs, data_dir / "qa_pairs.json")


In [7]:
qa_pairs = load_qa_pairs(data_dir / "qa_pairs.json")
print(qa_pairs)

[QuestionAnswerPair(question=AIMessage(content='How does Bitcoin allow online payments to be sent directly from one party to another without the need for a financial institution?', additional_kwargs={}, response_metadata={}), source_chunk='Bitcoin: A Peer-to-Peer Electronic Cash System\nSatoshi Nakamoto\nsatoshin@gmx.com\nwww.bitcoin.org\nAbstract.  A purely peer-to-peer version of electronic cash would allow online \npayments to be sent directly from one party to another without going through a \nfinancial institution.  Digital signatures provide part of the solution, but the main \nbenefits are lost if a trusted third party is still required to prevent double-spending', chunk_index=0, retrieved_chunks=None, answer=None), QuestionAnswerPair(question=AIMessage(content='How does using a peer-to-peer network eliminate the need for a trusted third party to prevent double-spending?', additional_kwargs={}, response_metadata={}), source_chunk='.  Digital signatures provide part of the soluti

In [6]:
from utils import calculate_metric_avg, evaluate_retrieval

k = 3
retriever = vector_store.as_retriever(k=k)

metrics = evaluate_retrieval(qa_pairs, retriever, k=k)
results = calculate_metric_avg(metrics)

print(metrics)
print(results)

{'precision': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'recall': [1, 1, 1, 1, 0, 1, 1, 1, 1, 1], 'mrr': [1.0, 1.0, 0.3333333333333333, 1.0, 0.25, 1.0, 1.0, 1.0, 1.0, 0.5]}
{'avg_precision': 0.3, 'avg_recall': 0.9, 'avg_mrr': 0.8083333333333332}


# 2. Evaluating Generation

1. Correctness: Response vs. Reference Answer
2. Relevance: Response vs. Input Question
3. Groundedness: Response vs. Retrieved Chunks
4. Retrieval Relevance: Retrieved Chunks vs. Input Question